In [1]:
# import dependencies
import tensorflow as tf

In [2]:
# check tf version
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.3.0


In [ ]:
# load mnist dataset & normalize to 0 - 1.0
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [15]:
# build ML model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [19]:
# predict using (untrained) model built (returns logits as model built w/o softmax OR sigmoid later wrt. last layer)
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.17463037,  0.8219978 , -0.3311332 ,  0.77387655,  0.87129116,
         0.30805418, -0.3584746 , -0.29912114, -0.37615827,  0.11226162]],
      dtype=float32)

In [27]:
# convert logits to probabilities (softmax returns probabilities SUM to 1 among all classes)
tf.nn.softmax(predictions).numpy()

array([[0.08920971, 0.17043556, 0.05379746, 0.1624282 , 0.17904742,
        0.10194299, 0.05234649, 0.05554749, 0.05142895, 0.08381577]],
      dtype=float32)

In [28]:
# define loss function & get loss for prediction
# https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()

2.2833416

In [29]:
# compile model 
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [30]:
# train 
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 915us/step - loss: 0.2973 - accuracy: 0.9135
Epoch 2/5
1875/1875 [==============================] - 2s 881us/step - loss: 0.1437 - accuracy: 0.9580
Epoch 3/5
1875/1875 [==============================] - 2s 901us/step - loss: 0.1063 - accuracy: 0.9685
Epoch 4/5
1875/1875 [==============================] - 2s 882us/step - loss: 0.0862 - accuracy: 0.9734
Epoch 5/5
1875/1875 [==============================] - 2s 887us/step - loss: 0.0749 - accuracy: 0.9769


In [31]:
# evaluate trained model using hold-out (val OR test) data set 
# return [loss, accuracy]
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0760 - accuracy: 0.9767


[0.07602409273386002, 0.9767000079154968]

In [32]:
# adding softmax layer on top of trained model to directly get probabilities 
# (softmax returns probabilities SUM to 1 among all classes); (sigmoid returns probabilities 0-1 for each individual class)
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [34]:
# predict using the existing model wrapped with a new softmax layer
res = probability_model(x_test[:5])

In [52]:
# import additional dependency
import numpy as np

# generate labels
labels = [i for i in range(10)]

# getting the predicted value relative to labels generated
for i in res:
    print(labels[np.argmax(i)], end=" ")

7 2 1 0 4 